##Loading of DrugProt dataset

Downloading and unzipping file

In [3]:
!wget https://zenodo.org/record/5042151/files/drugprot-gs-training-development.zip
!unzip drugprot-gs-training-development.zip

--2022-11-10 15:05:32--  https://zenodo.org/record/5042151/files/drugprot-gs-training-development.zip
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3796908 (3.6M) [application/octet-stream]
Saving to: ‘drugprot-gs-training-development.zip’

drugprot-gs-trainin 100%[===================>]   3.62M  --.-KB/s    in 0.1s    

2022-11-10 15:05:33 (28.9 MB/s) - ‘drugprot-gs-training-development.zip’ saved [3796908/3796908]

Archive:  drugprot-gs-training-development.zip
   creating: drugprot-gs-training-development/
  inflating: drugprot-gs-training-development/README.pdf  
   creating: drugprot-gs-training-development/development/
  inflating: drugprot-gs-training-development/development/drugprot_development_relations.tsv  
  inflating: drugprot-gs-training-development/development/drugprot_development_entities.tsv  
  inflating: drugprot-gs-training-developmen

Combining all three files together.

In [4]:
def loadDrugProt(abstracts_filename, entities_filename, relations_filename):

  docs = {}

  # Load the title/abstracts text in as documents
  with open(abstracts_filename, encoding='utf8') as f:
    for lineno,line in enumerate(f):
      split = line.strip('\n').split('\t')
      assert len(split) == 3, f"Expected 3 columns but got {len(split)} on line {lineno+1}"
      pubmed_id, title, abstract = split
      pubmed_id = int(pubmed_id)

      combined_text = title + '\n' + abstract
      docs[pubmed_id] = {'pubmed_id':pubmed_id, 'text':combined_text, 'entities':{}, 'relations':[]}

  # Load the entities and match them up with the documents
  with open(entities_filename, encoding='utf8') as f:
    for lineno,line in enumerate(f):
      split = line.strip('\n').split('\t')
      assert len(split) == 6, f"Expected 6 columns but got {len(split)} on line {lineno+1}"
      pubmed_id, entity_id, entity_type, start_coord, end_coord, entity_text = split

      pubmed_id = int(pubmed_id)
      start_coord = int(start_coord)
      end_coord = int(end_coord)

      assert pubmed_id in docs, f"Did not find matching document for pubmed_id={pubmed_id}"
      doc = docs[pubmed_id]

      assert doc['text'][start_coord:end_coord] == entity_text, f"Text for entity with coords {start_coord}:{end_coord} in document (pubmed_id={pubmed_id} does not match expected. 'f{doc['text'][start_coord:end_coord]}' != '{entity_text}'"

      entity = {'type':entity_type, 'start':start_coord, 'end':end_coord, 'text':entity_text}
      doc['entities'][entity_id] = entity

  if relations_filename is not None:
    # Load the relations and match them up with the entities in the corresponding document
    with open(relations_filename, encoding='utf8') as f:
      for lineno,line in enumerate(f):
        split = line.strip('\n').split('\t')
        assert len(split) == 4, f"Expected 4 columns but got {len(split)} on line {lineno+1}"

        pubmed_id, relation_type, arg1, arg2 = split

        pubmed_id = int(pubmed_id)
        assert arg1.startswith('Arg1:'), f"Relation argument should start with 'Arg1:'. Got: {arg1}"
        assert arg2.startswith('Arg2:'), f"Relation argument should start with 'Arg2:'. Got: {arg2}"

        # Remove arg1/arg2 from text
        arg1 = arg1.split(':')[1]
        arg2 = arg2.split(':')[1]

        assert pubmed_id in docs, f"Did not find matching document for pubmed_id={pubmed_id}"
        doc = docs[pubmed_id]

        assert arg1 in doc['entities'], f"Couldn't find entity with id={arg1} in document with pubmed_id={pubmed_id}"
        assert arg2 in doc['entities'], f"Couldn't find entity with id={arg2} in document with pubmed_id={pubmed_id}"

        relation = {'type':relation_type,'arg1':arg1,'arg2':arg2}
        doc['relations'].append(relation)

  # Convert the dictionary of documents (indexed by pubmed_id) into a simpler dictionary
  docs = sorted(docs.values(), key=lambda x:x['pubmed_id'])

  return docs

Applying function to dataset

In [32]:
docs = loadDrugProt('drugprot-gs-training-development/training/drugprot_training_abstracs.tsv',
                    'drugprot-gs-training-development/training/drugprot_training_entities.tsv',
                    'drugprot-gs-training-development/training/drugprot_training_relations.tsv')

Observation of doc structure


In [33]:
search = [ d for i,d in enumerate(docs) if d['pubmed_id'] == 1280065]
assert len(search) == 1, "Something went wrong and couldn't find the document we want"

doc = search[0]
doc

{'pubmed_id': 1280065,
 'text': 'Analysis of plasmin binding and urokinase activation of plasminogen bound to the Heymann nephritis autoantigen, gp330.\nPreviously, we demonstrated that the Heymann nephritis autoantigen, gp330, can serve as a receptor site for plasminogen. This binding was not significantly inhibited by the lysine analogue epsilon-amino caproic acid (EACA), indicating that plasminogen binding was not just through lysine binding sites as suggested for other plasminogen binding sites. We now report that once plasminogen is bound to gp330, it can be converted to its active form of plasmin by urokinase. This conversion of plasminogen to plasmin proceeds at a faster rate when plasminogen is first prebound to gp330. Although there is a proportional increase in the Vmax of the urokinase-catalyzed reaction with increasing gp330 concentrations, no change in Km was observed. Once activated, plasmin remains bound to gp330 in an active state capable of cleaving the chromogenic tri

Total number of relations

In [36]:
num = 0
for doc in docs:
  num += len(doc['relations'])

print(num)

17288


## Encoding Step

In [7]:
#!pip install torch
!pip install transformers
!pip install segtok

from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import nltk
import torch
from segtok.segmenter import split_single

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 12.8 MB/s 
     |████████████████████████████████| 163 kB 64.8 MB/s 
     |████████████████████████████████| 7.6 MB 43.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Adding in start and end entity tags to biomedical text, e.g. [E1] CHEMICAL [/E1] and [E2] GENE [/E2]

In [24]:
def relations(text):
  # Make it so that it goes through the relations dictionary and extracts the specific terms within the relation
  pass

def addEntityTags(docs):

  for doc in docs:
    str = ''
    index = 0
    entity_dict = {}
    n = 1
    keys = doc['entities'].keys()
    key_list = sorted(keys, key= lambda x: (doc['entities'][x]['start']))
    for k in key_list:
      start = doc['entities'][k]['start']
      end = doc['entities'][k]['end']
      name = doc['entities'][k]['text']
      if entity_dict.get(name) is not None:
        i = entity_dict[name]
      else:
        entity_dict[name] = n
        n+=1
        i = entity_dict[name]
      str += doc['text'][index:start] + f'[E{i}] ' + doc['text'][start:end] + f' [/E{i}]'
      index = end
    doc['tagged_text'] = str 

addEntityTags(docs)
print(docs[0]['tagged_text'])
      

Analysis of [E1] plasmin [/E1] binding and [E2] urokinase [/E2] activation of [E3] plasminogen [/E3] bound to the [E4] Heymann nephritis autoantigen [/E4], [E5] gp330 [/E5].
Previously, we demonstrated that the [E4] Heymann nephritis autoantigen [/E4], [E5] gp330 [/E5], can serve as a receptor site for [E3] plasminogen [/E3]. This binding was not significantly inhibited by the [E6] lysine [/E6] analogue [E7] epsilon-amino caproic acid [/E7] ([E8] EACA [/E8]), indicating that [E3] plasminogen [/E3] binding was not just through [E6] lysine [/E6] binding sites as suggested for other [E3] plasminogen [/E3] binding sites. We now report that once [E3] plasminogen [/E3] is bound to [E5] gp330 [/E5], it can be converted to its active form of [E1] plasmin [/E1] by [E2] urokinase [/E2]. This conversion of [E3] plasminogen [/E3] to [E1] plasmin [/E1] proceeds at a faster rate when [E3] plasminogen [/E3] is first prebound to [E5] gp330 [/E5]. Although there is a proportional increase in the Vmax o

Reducing the text only to sentences potentially containing relations between chemicals and genes:

In [22]:
def relevantText():
  texts = [doc['tagged_text'] for doc in docs]

  split_text = []
  target_text = []

  for d in texts:
    split_text += split_single(d)

  for s in split_text:
    if '[/E' in s:
      target_text.append(s)

  return target_text

target_text = relevantText()

There are around 17288 total relations, therefore there some sentences that contain entities but don't actually contain a relation

In [23]:
print(len(target_text))

29234


Loading pre-trained PubMedBERT model + tokenizer

In [26]:
model = BertModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
                                  output_hidden_states = True,
                                  )

tokenizer = BertTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext") 

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Adding more tags and tokenising input

In [27]:
def bert_text_prep(text, tokenizer):

  #CLS lets BERT know when start of sentence begins, SEP indicates start of second sentence.
  marked_text = "[CLS]" + text + "[SEP]"

  #Tokenizes text
  tokenized_text = tokenizer.tokenize(marked_text)

  #Converts to ids
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [1]*len(indexed_tokens)

  # Converts inputs to tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  return tokenized_text, tokens_tensor, segments_tensors

Converting input into embeddings

In [28]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2][1:]

  token_embeddings = hidden_states[-1]
  #Collapsing tensor into 1 dimension
  token_embeddings = torch.squeeze(token_embeddings, dim=0)
  #converting tensors to lists
  list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

  return list_token_embeddings

Getting embeddings for word in all given contexts

In [30]:
target_word_embeddings = []
test = target_text[:2]

for text in test:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_prep(text, tokenizer)
    print(tokenized_text[:5])
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)

    target_word_embeddings.append(list_token_embeddings)

['[CLS]', 'analysis', 'of', '[', 'e1']
['[CLS]', 'previously', ',', 'we', 'demonstrated']


## Classification Step

Establishing the relation labels

In [ ]:
labels = list(set([r['type'] for doc in docs for r in doc['relations']]))

Finding all of the entity start tokens

In [ ]:
def findEmbeddings():
  pass